In [ ]:
import pygame, sys
import math
import random as rn
import numpy as np
import copy
import matplotlib.pyplot as plt
from scipy import interpolate
%matplotlib inline
plt.rcParams.update({'figure.figsize': [8,5]})

sys.path.append('procedural_tracks/')
from procedural_tracks.main import *

np.set_printoptions(precision=3, suppress=True)

In [ ]:
# helper functions

def get_track_points_from_random_track(seed=8, sample=20):
    rn.seed(seed)  
    points = random_points()
    hull = ConvexHull(points)
    track_points_coarse = shape_track(get_track_points(hull, points))
    corner_points = get_corners_with_kerb(track_points_coarse)
    track_points = smooth_track(track_points_coarse)[::sample]
    return track_points

def racing_line_length(points): 
    x,y = zip(*points)
    num_points = len(points)
    length = np.sum([np.sqrt((x[i] - x[i+1])**2 + (y[i] - y[i+1])**2) 
                     for i in range(num_points-1)])
    return length

# this is slightly incorrect, the new point can be (max_offset, max_offset) away
# use a 2d gaussian instead?
def get_new_point(old_point, baseline, spread, max_offset_from_baseline):
    old_x, old_y = old_point
    baseline_x, baseline_y = baseline
    while True:
        potential_x = old_x + (np.random.random()-0.5) * spread
        if abs(potential_x - baseline_x) < max_offset_from_baseline:
            new_x = potential_x
            break
    while True:
        potential_y = old_y + (np.random.random()-0.5) * spread
        if abs(potential_y - baseline_y) < max_offset_from_baseline:
            new_y = potential_y
            break
    return new_x, new_y

In [ ]:
# cost functions

def racing_line_length_full(points_except_last, params={}): 
    points = copy.deepcopy(points_except_last)
    points += [points[0]]
    x,y = zip(*points)
    num_points = len(points)
    length = np.sum([np.sqrt((x[i] - x[i+1])**2 + (y[i] - y[i+1])**2) 
                     for i in range(num_points-1)])
    return length

def _angle_between(vector1, vector2):
        vector1_unit = vector1 / np.linalg.norm(vector1)
        vector2_unit = vector2 / np.linalg.norm(vector2)
        return np.arccos(np.clip(np.dot(vector1_unit, vector2_unit), -1.0, 1.0))    
    
def sum_of_angles(points_except_last, params={}):
    num_points = len(points_except_last)
    points = copy.deepcopy(points_except_last)
    points += [points[0]]
    
    vectors = [(points[i+1][0] - points[i][0], points[i+1][1] - points[i][1]) for i in range(num_points)]
    vectors += [vectors[0]]
    angles = [_angle_between(vectors[i], vectors[i+1]) for i in range(len(vectors)-1)]
    return np.std(angles), angles
    
def _length_and_angles(points_except_last):
#     length = racing_line_length_full(points_except_last)
#     angles = sum_of_angles(points_except_last)
    points = copy.deepcopy(points_except_last)
    points += [points[0]]
    x,y = zip(*points)
    num_points = len(points)
    vectors = [(x[i+1] - x[i], y[i+1] - y[i]) for i in range(num_points-1)]
    vectors += [vectors[0]]
    
    length = np.sum([np.sqrt(vectors[i][0]**2 + vectors[i][1]**2) for i in range(len(vectors)-1)])
    angles = np.std([_angle_between(vectors[i], vectors[i+1]) for i in range(len(vectors)-1)])
    
    return (length, angles)

def length_and_angles(points_except_last, params={'reg_param': 0.0, 'init_cost': (1.0, 1.0)}):
    reg_param = params['reg_param']
    init_cost = params['init_cost']
    
    length, angles = _length_and_angles(points_except_last)
    length_init, angles_init = init_cost
    length_normalized = length / length_init
    angles_normalized = angles / angles_init
    
    cost = length_normalized + reg_param * angles_normalized
    return cost

In [ ]:
# optimization functions

def opt_random_perturbation(track_midpoints, compute_cost=racing_line_length_full,
                            spread=10, iterations=100, track_width=0.65,
                            debug=False, debug_iter=1000):
    
    best_racing_line = []
    best_racing_line_cost = np.inf
    num_track_points = np.array(track_midpoints).shape[0]
    max_offset_from_baseline = track_width/2
    racing_line = [0] * (num_track_points)
    
    for iteration in range(iterations):
        for i in range(num_track_points):
            new_x, new_y = get_new_point(track_midpoints[i], track_midpoints[i], 
                                         spread, max_offset_from_baseline)
            racing_line[i] = (new_x, new_y)
        cost = compute_cost(racing_line)
        if cost < best_racing_line_cost:
            best_racing_line = copy.deepcopy(racing_line)
            best_racing_line_cost = cost
        if debug:
            if iteration % debug_iter == 0:
                print(f'Best hiterto: {best_racing_line_cost:.3f}')
    return best_racing_line, best_racing_line_cost

def opt_iterative_improvement(track_midpoints, compute_cost=racing_line_length_full,
                              cost_func_params={},
                              spread=10, iterations=100, track_width=0.65,
                              debug=False, debug_iter=1000):
    
    num_track_points = np.array(track_midpoints).shape[0]
    racing_line_old = copy.deepcopy(track_midpoints)
    racing_line_new = copy.deepcopy(racing_line_old)
    max_offset_from_baseline = track_width/2
    init_cost = compute_cost(racing_line_old)
    print(f'Started with: {init_cost}')
    min_cost_so_far = init_cost
    
    for iteration in range(iterations):
        min_cost_iter = min_cost_so_far
        point = (-1, (0,0))
        for i in range(num_track_points):
            new_x, new_y = get_new_point(racing_line_old[i], track_midpoints[i], 
                                         spread, max_offset_from_baseline)
            racing_line_new[i] = (new_x, new_y)
            cost = compute_cost(racing_line_new, cost_func_params)
            if cost < min_cost_iter:
                point = (i, (new_x, new_y))
                min_cost_iter = cost
            racing_line_new[i] = racing_line_old[i]
        if min_cost_iter < min_cost_so_far:
            racing_line_new[point[0]] = point[1]
            min_cost_so_far = min_cost_iter
        if debug:
            if iteration % debug_iter == 0:
                print(f'Smallest hiterto: {min_cost_so_far:.3f}')
                #print(np.array(racing_line_new))
        racing_line_old = copy.deepcopy(racing_line_new)
    
    print(f'\nEnded with: {compute_cost(racing_line_new)}\n')
    return racing_line_new

In [ ]:
# visualization functions

def draw_track_and_racing_line(track_points, racing_line, track_width, 
                               compute_cost=racing_line_length_full,
                               xlims=None, ylims=None, smooth=False, smooth_points=1000):
    track_points_x, track_points_y = zip(*track_points)
    plt.scatter(track_points_x, track_points_y, s=1000*track_width, color='black', facecolor='None')
    if xlims != None:
        plt.xlim((-2, 6))
    if ylims != None:
        plt.ylim((-2, 6))
#     print(compute_cost(racing_line))
    racing_line = copy.deepcopy(racing_line)
    racing_line += [racing_line[0]]
    x, y = zip(*racing_line)
    plt.plot(x, y, 'or')
    if smooth:
        tck, u = interpolate.splprep([x, y], s=0, per=True)
        xi, yi = interpolate.splev(np.linspace(0, 1, smooth_points), tck)
        plt.plot(xi, yi, '-b')
    else:
        plt.plot(x, y, '-b')
    #print(f'Length of racing line: {racing_line_length(racing_line):.3f}')
    
def draw_complete_track(track_seed=8, racing_line=None):
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    background_color = GRASS_GREEN
    screen.fill(background_color)
    
    rn.seed(track_seed)  # 8 is a nice simple track
    points = random_points()
    hull = ConvexHull(points)
    track_points = shape_track(get_track_points(hull, points))
    corner_points = get_corners_with_kerb(track_points)
    f_points = smooth_track(track_points)
    # get complete corners from keypoints
    corners = get_full_corners(f_points, corner_points)
    # draw the actual track (road, kerbs, starting grid)
    draw_track(screen, GREY, f_points, corners)
    # draw racing line
    draw_lines_from_points(screen, BLUE, racing_line)
    # draw checkpoints
    checkpoints = get_checkpoints(f_points)

    pygame.display.set_caption(TITLE)
    while True: # main loop
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
        pygame.display.update()

### Loading and checking a sample track

In [ ]:
rn.seed(8)  # 8 is a nice simple track
points = random_points()
hull = ConvexHull(points)
track_points = shape_track(get_track_points(hull, points))
corner_points = get_corners_with_kerb(track_points)
f_points = smooth_track(track_points)
# get complete corners from keypoints
corners = get_full_corners(f_points, corner_points)

In [ ]:
len(f_points)

In [ ]:
track_midpoints = np.array(f_points)
track_midpoints_x, track_midpoints_y = zip(*track_midpoints)
ax=plt.scatter(track_midpoints_x[::20], track_midpoints_y[::20], s=200, color='black', facecolor='None')

## Sanity checks on smaller tracks

In [ ]:
track_points = [[0,0], [4,0], [4,4], [0,4]]
track_width = 2

In [ ]:
draw_track_and_racing_line(track_points, track_points, track_width, xlims=(-2,6), ylims=(-2,6))
length_init, angles_init = _length_and_angles(track_points)
print(length_init, angles_init)

In [ ]:
draw_track_and_racing_line(track_points, track_points, track_width, xlims=(-2,6), ylims=(-2,6), smooth=True)

In [ ]:
# Compute shortest racing line using the random-perturbation algorithm

shortest_racing_line, _ = opt_random_perturbation(track_points, spread=0.8, iterations=100000)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width, xlims=(-2,6), ylims=(-2,6))

In [ ]:
# Compute shortest racing line using the iterative-improvement algorithm

shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=racing_line_length_full,
                                                 spread=0.8, iterations=1000,
                                                 debug=False, debug_iter=1)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width, xlims=(-2,6), ylims=(-2,6))

In [ ]:
# Compute shortest racing line using the iterative-improvement algorithm
init_cost = _length_and_angles(track_points)
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=length_and_angles,
                                                 cost_func_params = {'reg_param': 0.5, 'init_cost': init_cost},
                                                 spread=0.8, iterations=1000,
                                                 debug=False, debug_iter=1)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width, xlims=(-2,6), ylims=(-2,6))

In [ ]:
# new track
track_points = [[0,0], [4,0], [4,4], [2,4], [2,2], [0,2]]

shortest_racing_line = opt_iterative_improvement(track_points, spread=0.8, iterations=1000,
                                                 debug=False, debug_iter=1)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width=2, xlims=(-2,6), ylims=(-2,6))

In [ ]:
# new track
track_points = [[0,0], [2,0], [4,0], [4,2], [4,4], [2,4], [2,2], [0,2]]

shortest_racing_line = opt_iterative_improvement(track_points, spread=0.8, iterations=1000,
                                                 debug=False, debug_iter=1)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width, xlims=(-2,6), ylims=(-2,6))

In [ ]:
# Compute shortest racing line using the iterative-improvement algorithm
init_cost = _length_and_angles(track_points)
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=length_and_angles,
                                                 cost_func_params = {'reg_param': 0.0, 'init_cost': init_cost},
                                                 spread=0.8, iterations=1000,
                                                 debug=False, debug_iter=1)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width, xlims=(-2,6), ylims=(-2,6))

Seems to be working well enough. Back to the larger track(s).

In [ ]:
rn.seed(8)  # 8 is a nice simple track
points = random_points()
hull = ConvexHull(points)
track_points_coarse = shape_track(get_track_points(hull, points))
corner_points = get_corners_with_kerb(track_points_coarse)
track_points = smooth_track(track_points_coarse)[::20]

In [ ]:
draw_track_and_racing_line(track_points, track_points, track_width=2)#, xlims=(-2,6), ylims=(-2,6))
print(length_and_angles(track_points))

In [ ]:
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=racing_line_length_full,
                                                 spread=70, iterations=1000,
                                                 debug=False, debug_iter=10, track_width=70)
draw_track_and_racing_line(track_points, shortest_racing_line, 
                           compute_cost=racing_line_length_full,
                           track_width=2)#, xlims=(-2,6), ylims=(-2,6))

Ooh la la

In [ ]:
# will open a new window, closing which throws the following (harmless) exception
draw_complete_track(racing_line=shortest_racing_line)

In [ ]:
# new track

track_points = get_track_points_from_random_track(seed=123, sample=20)
draw_track_and_racing_line(track_points, track_points, track_width=2)

In [ ]:
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=racing_line_length_full,
                                                 spread=70, iterations=1000,
                                                 debug=False, debug_iter=10, track_width=70)
draw_track_and_racing_line(track_points, shortest_racing_line, 
                           compute_cost=racing_line_length_full, track_width=2)

In [ ]:
# new track

track_points = get_track_points_from_random_track(seed=95521, sample=20)
draw_track_and_racing_line(track_points, track_points, track_width=2, smooth=True, smooth_points=1000)
print(_length_and_angles(track_points))

In [ ]:
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=racing_line_length_full,
                                                 spread=70, iterations=1000,
                                                 debug=False, debug_iter=10, track_width=70)
draw_track_and_racing_line(track_points, shortest_racing_line, 
                           compute_cost=racing_line_length_full, track_width=2)
print(_length_and_angles(shortest_racing_line))

In [ ]:
draw_track_and_racing_line(track_points, shortest_racing_line, 
                           compute_cost=racing_line_length_full, track_width=2, smooth=False, smooth_points=2000)
print(_length_and_angles(shortest_racing_line))

### In quest for a more realistic cost function

In [ ]:
# Compute shortest racing line using the iterative-improvement algorithm
init_cost = _length_and_angles(track_points)
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=length_and_angles,
                                                 cost_func_params = {'reg_param': 1, 'init_cost': init_cost},
                                                 spread=70, iterations=1000,
                                                 debug=True, debug_iter=50, track_width=70)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width=2)
print(_length_and_angles(shortest_racing_line))

In [ ]:
# Compute shortest racing line using the iterative-improvement algorithm
init_cost = _length_and_angles(track_points)
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=length_and_angles,
                                                 cost_func_params = {'reg_param': 0.5, 'init_cost': init_cost},
                                                 spread=70, iterations=1000,
                                                 debug=True, debug_iter=50, track_width=70)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width=2)
print(_length_and_angles(shortest_racing_line))

In [ ]:
# Compute shortest racing line using the iterative-improvement algorithm
init_cost = _length_and_angles(track_points)
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=length_and_angles,
                                                 cost_func_params = {'reg_param': 0.1, 'init_cost': init_cost},
                                                 spread=70, iterations=1000,
                                                 debug=True, debug_iter=50, track_width=70)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width=2)
print(_length_and_angles(shortest_racing_line))

In [ ]:
# Compute shortest racing line using the iterative-improvement algorithm
init_cost = _length_and_angles(track_points)
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=length_and_angles,
                                                 cost_func_params = {'reg_param': 0.03, 'init_cost': init_cost},
                                                 spread=70, iterations=1000,
                                                 debug=True, debug_iter=50, track_width=70)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width=2)
print(_length_and_angles(shortest_racing_line))

In [ ]:
# Compute shortest racing line using the iterative-improvement algorithm
init_cost = _length_and_angles(track_points)
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=length_and_angles,
                                                 cost_func_params = {'reg_param': 0.01, 'init_cost': init_cost},
                                                 spread=70, iterations=1000,
                                                 debug=True, debug_iter=50, track_width=70)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width=2)
print(_length_and_angles(shortest_racing_line))

In [ ]:
# Compute shortest racing line using the iterative-improvement algorithm
init_cost = _length_and_angles(track_points)
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=length_and_angles,
                                                 cost_func_params = {'reg_param': 0.02, 'init_cost': init_cost},
                                                 spread=70, iterations=1000,
                                                 debug=True, debug_iter=50, track_width=70)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width=2)
print(_length_and_angles(shortest_racing_line))

In [ ]:
# Compute shortest racing line using the iterative-improvement algorithm
init_cost = _length_and_angles(track_points)
shortest_racing_line = opt_iterative_improvement(track_points, compute_cost=length_and_angles,
                                                 cost_func_params = {'reg_param': 0.015, 'init_cost': init_cost},
                                                 spread=70, iterations=1000,
                                                 debug=True, debug_iter=50, track_width=70)
draw_track_and_racing_line(track_points, shortest_racing_line, track_width=2)
print(_length_and_angles(shortest_racing_line))

In [ ]:
draw_track_and_racing_line(track_points, shortest_racing_line, track_width=2, smooth=True, smooth_points=1000)